In [ ]:
import spacy
!python -m spacy download en_core_web_sm

In [8]:
import json
import spacy

with open("D:\Datascience_workspace_2023\MY_NLP\spacy-codes\data\countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

nlp = spacy.blank("en")
doc = nlp("Czech Republic may help Slovakia protect its airspace")

# Import the PhraseMatcher and initialize it
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

# Create pattern Doc objects and add them to the matcher
# This is the faster version of: [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", patterns)

# Call the matcher on the test document and print the result
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

[Czech Republic, Slovakia]


In [10]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
import json

with open("D:\Datascience_workspace_2023\MY_NLP\spacy-codes\data\countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())
with open("D:\Datascience_workspace_2023\MY_NLP\spacy-codes\data\country_text.txt", encoding="utf8") as f:
    TEXT = f.read()

nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", patterns)

# Create a doc and reset existing entities
doc = nlp(TEXT)
doc.ents = []

# Iterate over the matches
for match_id, start, end in matcher(doc):
    # Create a Span with the label for "GPE"
    span = Span(doc, start, end, label="GPE")

    # Overwrite the doc.ents and add the span
    doc.ents = list(doc.ents) + [span]

    # Get the span's root head token
    span_root_head = span.root.head
    # Print the text of the span root's head token and the span text
    print(span_root_head.text, "-->", span.text)

# Print the entities in the document
print([(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "GPE"])

in --> Namibia
in --> South Africa
Africa --> Cambodia
of --> Kuwait
as --> Somalia
Somalia --> Haiti
Haiti --> Mozambique
in --> Somalia
for --> Rwanda
Britain --> Singapore
War --> Sierra Leone
of --> Afghanistan
invaded --> Iraq
in --> Sudan
of --> Congo
earthquake --> Haiti
[('Namibia', 'GPE'), ('South Africa', 'GPE'), ('Cambodia', 'GPE'), ('Kuwait', 'GPE'), ('Somalia', 'GPE'), ('Haiti', 'GPE'), ('Mozambique', 'GPE'), ('Somalia', 'GPE'), ('Rwanda', 'GPE'), ('Singapore', 'GPE'), ('Sierra Leone', 'GPE'), ('Afghanistan', 'GPE'), ('Iraq', 'GPE'), ('Sudan', 'GPE'), ('Congo', 'GPE'), ('Haiti', 'GPE')]


In [ ]:
import spacy
from spacy.matcher import Matcher

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Define the modified drug pattern
pattern = [{"TEXT": "O", "IS_SPACE": True, "OP": "?"}, {"TEXT": "N", "IS_SPACE": True, "OP": "?"}, {"TEXT": "1", "IS_SPACE": True, "OP": "?"}, {"TEXT": "5", "IS_SPACE": True, "OP": "?"}, {"TEXT": "3", "IS_SPACE": True, "OP": "?"}]

# Initialize the Matcher with the current NLP model
matcher = Matcher(nlp.vocab)

# Add the drug pattern to the matcher
matcher.add("DrugMatcher", pattern)

# Process the text
text = "The patient received O N 15 and ON-15 medications."
doc = nlp(text)

# Use the matcher on the processed text
matches = matcher(doc)

# Extract matched spans from the document
drug_spans = [doc[start:end] for _, start, end in matches]

# Print the matched drug spans
for span in drug_spans:
    print(span.text)


In [2]:
import spacy
from spacy.matcher import Matcher

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Initialize the matcher
matcher = Matcher(nlp.vocab)

# Define a sequence of patterns
pattern_sequence = [
    {"LOWER": "start"},
    {"POS": "ADJ"},
    {"POS": "NOUN"},
    {"LOWER": "end"}
]

# Add the pattern sequence to the matcher
matcher.add("PatternSequence", pattern_sequence)

# Example text
text = "The journey starts with a beautiful sunrise and ends with a warm embrace."

# Process the text
doc = nlp(text)

# Apply the matcher to find matches
matches = matcher(doc)

# Extract matched spans and entities
matched_sequences = []
for match_id, start, end in matches:
    label = nlp.vocab.strings[match_id]
    sequence_text = doc[start:end].text
    matched_sequences.append((sequence_text, label))

# Print the identified pattern sequences
print(matched_sequences)


ValueError: [E178] Each pattern should be a list of dicts, but got: {'LOWER': 'start'}. Maybe you accidentally passed a single pattern to Matcher.add instead of a list of patterns? If you only want to add one pattern, make sure to wrap it in a list. For example: `matcher.add('PatternSequence', [pattern])`

Introduction to spaCy

Hi, I'm Ines! I'm one of the core developers of spaCy, a popular library for advanced Natural Language Processing in Python.

In this lesson, we'll take a look at the most important concepts of spaCy and how to get started.

In [5]:
 
# Import spaCy
import spacy

# Create a blank English nlp object
nlp = spacy.blank("en")
# contains the processing pipeline
# includes language-specific rules for tokenization etc.
# The Doc object

# Created by processing a string of text with the nlp object
doc = nlp("Hello world!")

for token in doc:
    print(token.text)




Hello
world
!


In [7]:
# Index into the Doc to get a single Token
token = doc[1]

# Get the token text via the .text attribute
print(token.text)


# Token objects represent the tokens in a document – for example, a word or a punctuation character.

# To get a token at a specific position, you can index into the doc.

# Token objects also provide various attributes that let you access more information about the tokens. For example, the .text attribute returns the verbatim token text.

world


In [9]:
import spacy

# Load the small English pipeline
nlp = spacy.load("en_core_web_sm")

# Process a text
doc = nlp("She ate the pizza")

# Iterate over the tokens
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_,token.dep_,token.head.text)

She PRON nsubj ate
ate VERB ROOT ate
the DET det pizza
pizza NOUN dobj ate


In [17]:
# Predicting Named Entities
# Process a text
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text,ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY


In [18]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

In [23]:
import spacy
from spacy.matcher import Matcher

 

nlp = spacy.load("en_core_web_sm")
doc = nlp("Upcoming iPhone X release date leaked as Apple reveals pre-orders")

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Create a pattern matching two tokens: "iPhone" and "X"
pattern = [{"TEXT": "iPhone"}, {"TEXT": "X"}]


# Add the pattern to the matcher
matcher.add("IPHONE_X_PATTERN", [pattern])

# Use the matcher on the doc
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: ['iPhone X']


In [ ]:
# Part 1
# Write one pattern that only matches mentions of the full iOS versions: “iOS 7”, “iOS 11” and “iOS 10”.

In [25]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "After making the iOS update you won't notice a radical system-wide "
    "redesign: nothing like the aesthetic upheaval we got with iOS 7. Most of "
    "iOS 11's furniture remains the same as in iOS 10. But you will discover "
    "some tweaks once you delve a little deeper."
)

# Write a pattern for full iOS versions ("iOS 7", "iOS 11", "iOS 10")
pattern = [{"TEXT": "iOS"}, {"IS_DIGIT": True}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("IOS_VERSION_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 3
Match found: iOS 7
Match found: iOS 11
Match found: iOS 10


In [27]:
# Write one pattern that only matches forms of “download” (tokens with the lemma “download”), followed by a token with the part-of-speech tag "PROPN" (proper noun).
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "i downloaded Fortnite on my laptop and can't open the game at all. Help? "
    "so when I was downloading Minecraft, I got the Windows version where it "
    "is the '.zip' folder and I used the default program to unpack it... do "
    "I also need to download Winzip?"
)

# Write a pattern that matches a form of "download" plus proper noun
pattern = [{"LEMMA": "download"}, {"POS": "PROPN"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("DOWNLOAD_THINGS_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 3
Match found: downloaded Fortnite
Match found: downloading Minecraft
Match found: download Winzip


In [29]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(
    "Features of the app include a beautiful design, smart search, automatic "
    "labels and optional voice responses."
)

# Write a pattern for adjective plus one or two nouns
pattern = [{"POS":"ADJ"  }, {"POS": "NOUN"}, {"POS": "NOUN", "OP": "?"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("ADJ_NOUN_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 5
Match found: beautiful design
Match found: smart search
Match found: automatic labels
Match found: optional voice
Match found: optional voice responses


In [ ]:
# Chapter 2: Large-scale data analysis with spaCy

In [36]:
# Shared vocab and string store (1)

import spacy

nlp =spacy.blank("en")
nlp.vocab.strings.add("coffee")
coffee_hash = nlp.vocab.strings["coffee"]
print(coffee_hash)
coffee_string =nlp.vocab.strings[coffee_hash]
print(coffee_string) 
coffee_string1 =nlp.vocab.strings[3197928453018144401]
print(coffee_string1) 

# Hashes can't be reversed – that's why we need to provide the shared vocab
# Raises an error if we haven't seen the string before
string = nlp.vocab.strings[3197928453018144401]

# Print the lexical attributes
print(lexeme.text, lexeme.orth, lexeme.is_alpha)

3197928453018144401
coffee
coffee


In [37]:
# exemes: entries in the vocabulary
# A Lexeme object is an entry in the vocabulary

doc = nlp("I love coffee")
lexeme = nlp.vocab["coffee"]

# Print the lexical attributes
print(lexeme.text, lexeme.orth, lexeme.is_alpha)

coffee 3197928453018144401 True


In [40]:
# The Doc object
# Create an nlp object
import spacy
nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)
 
print(doc)

Hello world!


In [49]:
# The Span object (2)
# Import the Doc and Span classes
from spacy.tokens import Doc, Span

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Create a span manually
span = Span(doc, 0, 2)

# Create a span with a label
span_with_label = Span(doc, 0, 2, label="GREETING")

# Add span to the doc.ents
doc.ents = [span_with_label]


for ent in doc.ents:
    print(ent.text,ent.label_)
    
    
    # The doc.ents are writable, so we can add entities manually by overwriting it with a list of spans.

Hello world GREETING


In [52]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "Go, get started!"
words = ["Go", ",", "get", "started", "!"]
spaces = [False, True, True, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Go, get started!


In [53]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "spaCy is cool!"
words = ["spaCy", "is", "cool", "!"]
spaces = [True, True, False, True]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

spaCy is cool! 


In [56]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "Oh, really?!"
words = ["Oh", ",", "really", "?", "!"]
spaces = [False, True, False, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Oh, really?!


In [59]:
import spacy

nlp = spacy.blank("en")

# Import the Doc and Span classes
from spacy.tokens import Doc, Span

words = ["I", "like", "David", "Bowie"]
spaces = [True, True, True, False]

# Create a doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

# Create a span for "David Bowie" from the doc and assign it the label "PERSON"
span = Span(doc, 2, 3, label="PERSON")
print(span.text, span.label_)

# Add the span to the doc's entities
doc.ents = [span]

# Print entities' text and labels
print([(ent.text, ent.label_) for ent in doc.ents])

I like David Bowie
David PERSON
[('David', 'PERSON')]


In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

doc1 = nlp("It's a warm summer day")
doc2 = nlp("It's sunny outside")

# Get the similarity of doc1 and doc2
similarity = doc1.similarity(doc2)
print(similarity)

In [65]:
# Recap: Rule-based Matching
# Initialize with the shared vocab
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

# Patterns are lists of dictionaries describing the tokens
pattern = [{"LEMMA": "love", "POS": "VERB"}, {"LOWER": "cats"}]
matcher.add("LOVE_CATS", [pattern])

# Operators can specify how often a token should be matched
pattern = [{"TEXT": "very", "OP": "+"}, {"TEXT": "happy"}]
matcher.add("VERY_HAPPY", [pattern])

# Calling matcher on doc returns list of (match_id, start, end) tuples
doc = nlp("I love cats and I'm very very happy")
matches = matcher(doc)

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Match found: love cats
Match found: very happy
Match found: very very happy


In [ ]:
# Patterns are lists of dictionaries, and each dictionary describes one token and its attributes. Patterns can be added to the matcher using the matcher.add method.

In [71]:
# Adding statistical predictions
matcher = Matcher(nlp.vocab)
matcher.add("DOG", [[{"LOWER": "golden"}, {"LOWER": "retriever"}]])
doc = nlp("I have a Golden Retriever")

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print("Matched span:", span.text)
    # Get the span's root token and root head token
    print("Root token:", span.root.text)
    print("Root head token:", span.root.head.text)
    # Get the previous token and its POS tag
    print("Previous token:", doc[start - 1].text, doc[start - 1].pos_)

Matched span: Golden Retriever
Root token: Retriever
Root head token: have
Previous token: a DET


In [ ]:
# Efficient phrase matching (1)
# PhraseMatcher like regular expressions or keyword search – but with access to the tokens!
# Takes Doc object as patterns
# More efficient and faster than the Matcher
# Great for matching large word lists

In [77]:
from spacy.matcher import PhraseMatcher 
matcher= PhraseMatcher(nlp.vocab)

pattern=nlp("Golden Retriever")
matcher.add("DOG",None,pattern)

doc = nlp("I have a Golden Retriever")

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print("matched span: ", span.text)

In [79]:
from spacy.matcher import PhraseMatcher
import spacy

nlp = spacy.load("en_core_web_sm")

matcher = PhraseMatcher(nlp.vocab)

# One target phrase
target = "Golden Retriever"

# Generate variations of the target phrase
variations = [
    target,
    target.lower(),
    target.upper(),
    target.capitalize(),
    target.replace(" ", ""),
    # Add more variations as needed
]

# Convert variations to spaCy Doc objects
pattern_docs = [nlp(variation) for variation in variations]

# Add the pattern to the matcher
matcher.add("DOG", None, *pattern_docs)

doc = nlp("I have a golden retriever, and my friend has a GoldenRetriever.")

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print("matched span:", span.text)


matched span: golden retriever
matched span: GoldenRetriever
